In [106]:
!pip install --quiet tensorflow-text
# Clone the entire repo.
%cd /content/
!rm -r -f dl4tm
!git clone git://github.com/Jeilef/DL4TM_Text2Python.git dl4tm
%cd dl4tm
!ls

/content
Cloning into 'dl4tm'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 85 (delta 41), reused 39 (delta 13), pack-reused 0
Receiving objects: 100% (85/85), 398.92 KiB | 7.25 MiB/s, done.
Resolving deltas: 100% (41/41), done.
/content/dl4tm
conala-corpus-v1.1  README.md	      text2python.ipynb  transformer_util.py
google.ipynb	    requirements.txt  transformer.ipynb


In [107]:
import numpy as np
import json
import logging
from tokenize import tokenize
from io import BytesIO
import tensorflow_hub as hub
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_text # See https://github.com/tensorflow/hub/issues/463

In [108]:
logging.getLogger('tensorflow').setLevel(logging.ERROR)  # suppress warnings

In [109]:
train_file = open("conala-corpus-v1.1/conala-corpus/conala-train.json", "r")
train_json = json.load(train_file)

print(len(train_json))
print(train_json[0])

2379
{'intent': 'How to convert a list of multiple integers into a single integer?', 'rewritten_intent': "Concatenate elements of a list 'x' of multiple integers to a single integer", 'snippet': 'sum(d * 10 ** i for i, d in enumerate(x[::-1]))', 'question_id': 41067960}


In [110]:
base_sentences = np.array([item["rewritten_intent"] for item in train_json])
codes = np.array([item["snippet"] for item in train_json])

print(base_sentences[0])
print(codes[0])

Concatenate elements of a list 'x' of multiple integers to a single integer
sum(d * 10 ** i for i, d in enumerate(x[::-1]))


In [111]:
# Use custom tokenize function for code snippets
def tokenize_python(s):
    tokenized = tokenize(BytesIO(s.numpy()).readline)
    tokens = [token[1] for token in tokenized]
    tensor = tf.convert_to_tensor(tokens, dtype=tf.string)
    
    return tensor

def detokenize_python(s):
  array = s.numpy()
  code = b"".join(array)
  code = code.encode("utf-8")
  return code

example = tf.constant(codes[0])
tokenize_python(example)

<tf.Tensor: shape=(26,), dtype=string, numpy=
array([b'utf-8', b'sum', b'(', b'd', b'*', b'10', b'**', b'i', b'for',
       b'i', b',', b'd', b'in', b'enumerate', b'(', b'x', b'[', b':',
       b':', b'-', b'1', b']', b')', b')', b'', b''], dtype=object)>

In [112]:
data = tf.data.Dataset.from_tensor_slices(codes)
data = data.map(lambda x : tf.py_function(func=tokenize_python, inp=[x], Tout=tf.string))

In [113]:
from keras.layers.experimental.preprocessing import StringLookup

def preprocess_codes(codes):
  tokenized_codes = [tf.py_function(func=tokenize_python, inp=[snippet], Tout=tf.string) for snippet in codes]
  vectorize_layer = StringLookup(max_tokens=output_tokens)
  vectorize_layer.adapt(data)
  vectorized_codes = [vectorize_layer(code) for code in tokenized_codes]
  return tf.keras.preprocessing.sequence.pad_sequences(vectorized_codes, value=0), vectorize_layer.get_vocabulary()

def postprocess_codes(codes, vocabulary):
  codes = codes.reshape((len(vocabulary) + 2,))
  word = np.argmax(codes)
  return vocabulary[word - 2], word # because start and stop are first predictions in first prediction
  #codes = []
  #tokenized_codes = [tf.py_function(func=tokenize_python, inp=[snippet], Tout=tf.string) for snippet in codes]
  #i_layer = StringLookup(vocabulary=vocabulary, invert=True)
  #vectorized_codes = [vectorize_layer(code) for code in tokenized_codes]
  #return tf.keras.preprocessing.sequence.pad_sequences(vectorized_codes, value=0)

tokenized_codes, vocabulary = preprocess_codes(codes)

In [114]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask

def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  return seq[:, tf.newaxis, tf.newaxis, :]


class DecoderMasking(tf.keras.layers.Layer):
  def call(self, targets):
    padded_targets = create_padding_mask(targets)
    look_ahead_mask = create_look_ahead_mask(tf.shape(targets)[1])
    return tf.maximum(padded_targets, look_ahead_mask)

class DecoderPaddingMasked(tf.keras.layers.Layer):
  def call(self, inputs):
    expanded_inputs = inputs[:, tf.newaxis, tf.newaxis, :]
    return tf.cast(expanded_inputs, tf.float32)

example = tf.constant(codes[0])
tokenized_example = tokenize_python(example)

vectorize_layer = keras.layers.experimental.preprocessing.StringLookup(max_tokens=8000)
vectorize_layer.adapt(data)

vectorized_example = vectorize_layer(tokenized_example)

masking = DecoderMasking()
lam = masking(tf.convert_to_tensor(([vectorized_example])))
print(lam.shape)

(1, 1, 26, 26)


In [115]:
# Pretrained BERT encoder
encoder_inputs = tf.keras.layers.Input(shape=(None,), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 128].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 128].

In [116]:
# Transformer Decoder  taken from https://www.tensorflow.org/tutorials/text/transformer
from transformer_util import Decoder
output_tokens = len(vocabulary) + 2 # start and stop token 
sample_decoder = Decoder(num_layers=2, d_model=128, num_heads=2,
                         dff=128, target_vocab_size=output_tokens,
                         maximum_position_encoding=5000)

decoder_input = tf.keras.layers.Input(shape=(None,), dtype=tf.int32)

# Apply masking layer here
look_ahead_mask = DecoderMasking()(decoder_input)
dec_padding_mask = DecoderPaddingMasked()(encoder_inputs['input_mask'])

decoder_output, attn = sample_decoder(decoder_input,
                              enc_output=sequence_output,
                              training=False,
                              look_ahead_mask=look_ahead_mask,
                              padding_mask=dec_padding_mask)
final_layer = tf.keras.layers.Dense(output_tokens)
output = final_layer(decoder_output)


embedding_model = tf.keras.Model((text_input, decoder_input), output)
embedding_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None,)]            0                                            
__________________________________________________________________________________________________
input_13 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
keras_layer_10 (KerasLayer)     {'input_mask': (None 0           input_1[0][0]                    
__________________________________________________________________________________________________
keras_layer_11 (KerasLayer)     {'pooled_output': (N 4385921     keras_layer_10[0][0]             
                                                                 keras_layer_10[0][1]       

In [117]:
sentence = tf.constant([train_json[0]["rewritten_intent"]])
snippet = tf.constant(train_json[0]["snippet"])
snippet = tf.py_function(func=tokenize_python, inp=[snippet], Tout=tf.string)
snippet = tf.expand_dims(snippet, axis=0)
print(sentence.shape, snippet.shape)
# result = embedding_model((sentence, snippet))
result

(1,) (1, 26)


<tf.Tensor: shape=(1, 26, 8000), dtype=float32, numpy=
array([[[ 0.25900948,  0.05902249,  0.00255737, ..., -0.14142132,
          0.060927  ,  0.23605488],
        [ 0.2641915 ,  0.04997048,  0.11746741, ..., -0.11563613,
          0.0299585 ,  0.2452411 ],
        [ 0.22051044, -0.03371699,  0.07480508, ..., -0.12167043,
          0.09683892,  0.18546578],
        ...,
        [ 0.21927257, -0.07914125,  0.10151628, ..., -0.08671107,
          0.06166898,  0.28820792],
        [ 0.1649797 , -0.09419025,  0.05277481, ..., -0.0641139 ,
          0.06363688,  0.22325364],
        [ 0.16321684, -0.08993292,  0.04195784, ..., -0.05346364,
          0.04414991,  0.2221666 ]]], dtype=float32)>

In [118]:
# TODO: Preprocess code samples
# TODO: Set up masking
# TODO: Run training

In [119]:
# Taken from https://www.tensorflow.org/tutorials/text/transformer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, 
                                     beta_2=0.98, epsilon=1e-9)
embedding_model.compile(optimizer, loss_function)

In [120]:
base_sentences = np.array(['' if v is None else v for v in base_sentences])

In [121]:
history = embedding_model.fit([base_sentences, tokenized_codes], tokenized_codes, batch_size=64, epochs=15)

Epoch 1/15
38/38 [==============================] - 74s 2s/step - loss: 6.3215
Epoch 2/15
38/38 [==============================] - 65s 2s/step - loss: 2.3900
Epoch 3/15
38/38 [==============================] - 64s 2s/step - loss: 1.1449
Epoch 4/15
38/38 [==============================] - 64s 2s/step - loss: 0.7157
Epoch 5/15
38/38 [==============================] - 69s 2s/step - loss: 0.5091
Epoch 6/15
38/38 [==============================] - 64s 2s/step - loss: 0.3611
Epoch 7/15
38/38 [==============================] - 65s 2s/step - loss: 0.2658
Epoch 8/15
38/38 [==============================] - 65s 2s/step - loss: 0.1925
Epoch 9/15
38/38 [==============================] - 64s 2s/step - loss: 0.1329
Epoch 10/15
38/38 [==============================] - 66s 2s/step - loss: 0.0865
Epoch 11/15
38/38 [==============================] - 64s 2s/step - loss: 0.0515
Epoch 12/15
38/38 [==============================] - 64s 2s/step - loss: 0.0275
Epoch 13/15
38/38 [==============================

In [123]:
def generate_result(test_sentence, model):
  prediction = ""
  words = [0]
  while len(prediction) < 100:
    # prediction_prep = tokenize_python(tf.constant([prediction]))
    word_pred = model.predict((np.array([test_sentence]), np.array(words)))
    result, word_idx = postprocess_codes(word_pred, vocabulary)

    if word_idx == 1:
      break
    elif word_idx == 0:
      prediction += "[UNK]"
    else:
      prediction += result
    words += word_idx
  return prediction

print(len(vocabulary))
print(generate_result("add 5 and 6.", embedding_model))
print(generate_result("read the file fun.txt.", embedding_model))
print(generate_result("Concatenate elements of a list 'x' of multiple integers to a single integer", embedding_model))

2840


InvalidArgumentError: ignored